## 練習題 3-4

請協助調整以下API 程式碼，讓使用者可以調用該API 服務輸入資料，並透過該API 服務進行預測


In [1]:
#@markdown 解壓縮 sample_model

! wget https://raw.githubusercontent.com/ywchiu/HPPY/main/model/sample_model.zip > /dev/null
! wget https://raw.githubusercontent.com/ywchiu/HPPY/main/model/encoder.pickle > /dev/null
! unzip sample_model.zip > /dev/null
! ls sample_model/

--2021-06-21 08:50:29--  https://raw.githubusercontent.com/ywchiu/HPPY/main/model/sample_model.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6843295 (6.5M) [application/zip]
Saving to: ‘sample_model.zip’

sample_model.zip    100%[===================>]   6.53M  5.35MB/s    in 1.2s    

2021-06-21 08:50:32 (5.35 MB/s) - ‘sample_model.zip’ saved [6843295/6843295]

--2021-06-21 08:50:32--  https://raw.githubusercontent.com/ywchiu/HPPY/main/model/encoder.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10190694 (9.7M) [appl

### 預測類別 Sentiment_prediciton

In [2]:
import tensorflow as tf
from  tensorflow.keras.models import load_model
import pickle

class Sentiment_prediciton(object):
  def __init__(self, model_name, encoder_name):
    self.model = load_model(model_name)
    with open(encoder_name, 'rb') as handle:
      self.encoder = pickle.load(handle)

  def pad_to_size(self, vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

  def sample_predict(self, sample_pred_text, pad):
    encoded_sample_pred_text = self.encoder.encode(sample_pred_text)

    if pad:
      encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = self.model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return (predictions)

In [3]:
predictor = Sentiment_prediciton('sample_model', 'encoder.pickle')
predictions = predictor.sample_predict(('this is the worst movie I have ever had'), pad=False)
print(predictions)

ModuleNotFoundError: No module named 'tensorflow_datasets'

### 請完成以下程式碼，讓使用者得以用 Flask 提供正負向情緒判斷服務

In [12]:
from  tensorflow.keras.models import load_model
from flask import Flask
from flask import jsonify
from flask import request
app = Flask(__name__)

@app.route("/submit", methods=['POST'])
def sentiment_anaysis():
  sample_text = request.values['text']
  predictions = ''
  return jsonify({'data':predictions})


In [ ]:
#@markdown 啟用服務器
import threading,sys
class server_thread_wrapper(threading.Thread): 
  def __init__(self, *args, **keywords): 
    threading.Thread.__init__(self, *args, **keywords) 
    self.killed = False
  def start(self): 
    self.__run_backup = self.run 
    self.run = self.__run       
    threading.Thread.start(self) 
  
  def __run(self): 
    sys.settrace(self.globaltrace) 
    self.__run_backup() 
    self.run = self.__run_backup 
  
  def globaltrace(self, frame, event, arg): 
    if event == 'call': 
      return self.localtrace 
    else: 
      return None
  
  def localtrace(self, frame, event, arg): 
    if self.killed: 
      if event == 'line': 
        raise SystemExit() 
    return self.localtrace 
  
  def kill(self): 
    self.killed = True

def start_server():
  server = server_thread_wrapper(target=app.run)
  server.start()
  return server

def kill_server():
  server.kill()
  server.join()

if server:
  kill_server()
## start the server
server = start_server()

In [15]:
#@markdown 檢視答案

import requests
payload = {'text': 'this is the worst movie I have ever had'}
res = requests.post('http://127.0.0.1:5000/submit', data = payload)
predictions = predictor.sample_predict(('this is the worst movie I have ever had'), pad=False)

if predictions == res.json()['data']:
  print('答案正確')
else:
  print('答案不正確歐，再想想還有哪邊要修改')
  


127.0.0.1 - - [21/Mar/2021 23:47:43] "POST /submit HTTP/1.1" 200 -


''